In [21]:
# IMPORTAÇÃO DE BIBLIOTECAS
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer




In [22]:
# Carregar o modelo spaCy para português
nlp = spacy.load('pt_core_news_sm')

# FUNÇÃO DE CARREGAR AS PALAVRAS CHAVES
def extract_keywords_spacy(text, num_keywords=10):
    # Processar o texto com spaCy
    doc = nlp(text)
    
    # Extrair tokens relevantes: substantivos, adjetivos, verbos e advérbios
    tokens = [token.text for token in doc if token.pos_ in ['NOUN', 'ADJ', 'VERB', 'ADV'] and not token.is_stop]
    
    # Usar TfidfVectorizer para encontrar os termos mais importantes
    vectorizer = TfidfVectorizer(max_features=num_keywords)
    tfidf_matrix = vectorizer.fit_transform([" ".join(tokens)])
    
    # Obter os nomes das características (palavras)
    feature_names = vectorizer.get_feature_names_out()
    
    # Obter as pontuações TF-IDF para cada termo
    tfidf_scores = tfidf_matrix.toarray().flatten()
    
    # Criar um dicionário de termos e suas pontuações
    keywords_scores = dict(zip(feature_names, tfidf_scores))
    
    # Classificar os termos pelo valor TF-IDF e selecionar os mais importantes
    sorted_keywords = Counter(keywords_scores).most_common(num_keywords)
    
    # Extrair apenas as palavras-chave (ignorando as pontuações)
    keywords = [keyword for keyword, score in sorted_keywords]
    
    return keywords

In [23]:
# FUNÇÃO DE PRÉ-PROCESSAMENTO
def pre_processamento(texto):
    # Processar o texto com spaCy
    doc = nlp(texto)
    
    # Filtrar tokens: remover stopwords, pontuações e tokens que não são letras
    tokens = [token for token in doc if not token.is_stop and not token.is_punct and token.is_alpha]
    
    # Lematizar os tokens filtrados
    lematizados = [token.lemma_.lower() for token in tokens]
    
    # Juntar os tokens lematizados em um único texto
    texto_limpo = " ".join(lematizados)
    
    return texto_limpo


In [24]:
data = pd.read_csv('novo1.csv')
data

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg
...,...,...,...,...
3995,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos
3996,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos
3997,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos
3998,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos


In [25]:
data.dropna(inplace=True)
data

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg
...,...,...,...,...
3995,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos
3996,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos
3997,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos
3998,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos


In [26]:
data.sentiment = data["sentiment"].replace(["neg", "pos"],[0, 1])

data.head()

/tmp/ipykernel_27067/4011907872.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.sentiment = data["sentiment"].replace(["neg", "pos"],[0, 1])


,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",0
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,0
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",0
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,0
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,0


In [27]:
print(data["sentiment"].value_counts())

sentiment
0    2000
1    2000
Name: count, dtype: int64


In [28]:
data.text_pt = data.text_pt.apply(pre_processamento)

In [29]:
def classificar_texto(texto, coluna_texto, coluna_classificacao):
    vetorizar = CountVectorizer(lowercase=True, max_features=50)
    bag_of_words = vetorizar.fit_transform(texto[coluna_texto])
    treino, teste, classe_treino, classe_teste = train_test_split(bag_of_words,
                                                              texto[coluna_classificacao],
                                                              random_state = 42)
    regressao_logistica = LogisticRegression(solver = "lbfgs")
    regressao_logistica.fit(treino, classe_treino)
    return regressao_logistica.score(teste, classe_teste)
print(classificar_texto(data, "text_pt", "sentiment"))

0.696


In [30]:
keys = data.text_pt.apply(extract_keywords_spacy)

data["keys"] = keys


In [31]:
data.head()


,id,text_en,text_pt,sentiment,keys
0,1,Once again Mr. Costner has dragged out a movie...,costner arrumar filme necessário terrível seqü...,0,"[costner, armário, fantasma, importar, persona..."
1,2,This is an example of why the majority of acti...,motivo filme ação mesmo genérico chato haver v...,0,"[filme, assistir, personagem, ação, bill, diab..."
2,3,"First of all I hate those moronic rappers, who...",odeiar raps imbecis poder agir tiver arma pres...,0,"[filme, único, agir, ator, gostar, haver, pers..."
3,4,Not even the Beatles could write songs everyon...,beatles escrever música gostar walter hill ser...,0,"[filme, ficar, gostar, atuação, cara, cinema, ..."
4,5,Brass pictures movies is not a fitting word fo...,filmes foto latão palavra apropriar verdade ou...,0,"[atraente, brass, italiano, material, qualidad..."
